# Single Path One-Shot Neural Architecture Search using Random Search

### Make everything a bit faster

Your task is to implement SPOS + Random Search to find the optimal ResNet-18-like architecture
for image classification on CIFAR-10 dataset.

### Proposed search space
The same search space as in the lecture, but without kernel size search.
This means that each search block should only have 3 operations.

### Latency awareness
While searching, you should implement hard constraint on model latency. As a proxy for
latency, use the number of MACs. The baseline ResNet-18 has 37M MACs for images of size 32x32.
The final selected architecture should have at most 30M MACs.
You can find an example computation for the number of MACs in this notebook.

### How to complete the task
Plan for your experiments:
1. Train baseline ResNet-18 model.  **(2 pts)**
2. Finalize supernet code. **(15 pts)**
3. Train supernet with the same hyperparameters except the number of epochs. You should increase
   it by a factor of 3-6. Save supernet weights.  **(10 pts)**
4. Write implementation for Random Search.  **(10 pts)**
5. Run it for 100-1000 iterations by measuring accuracies of models from the supernet.
   Keep track of each model accuracy and latency.  **(5 pts)**
6. Train the best architecture from scratch. For this, you can build the whole supernet, and
   select this architecture for all the forward passes of the training.  **(6 pts)**
7. Compare with the model from step 1. Does your model have a better accuracy?  **(2 pts)**

### Code structure

Main.ipynb - Learn your neural networks here.

resnet.py - ResNet-18 implementation from torchvision, simplified for this project.

supernet.py - Unfinished Supernet implementation.

cifar10.py - Transforms for CIFAR-10 training.


# Train supernet

## Setup

### Imports

In [1]:
from typing import Tuple

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

from cifar10 import get_train_transform, get_val_transform

### Make everything a bit faster

In [2]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

### Build datasets and dataloaders for CIFAR-10

In [3]:
# Change this value if needed.
batch_size = 512

In [4]:
train_transform = get_train_transform()
val_transform = get_val_transform()

train_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=train_transform,
)
test_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=val_transform,
)

train_dataloader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    drop_last=True,
)
test_dataloader = DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    drop_last=False,
)

Files already downloaded and verified
Files already downloaded and verified


## Create supernet

**Before running code in this section, you need to finish supernet implementation.**

Please, go to `supernet.py` file and inspect the current implementation of SearchBlock and Supernet classes.
Pay attention to the TODOs. You need to implement all of them.

Supernet and BasicBlock classes are modified versions of ResNet and BasicBlock classes from `resnet.py`.

    Tip: to understand how the Supernet is constructed, compare the implementation of Supernet and ResNet classes. You should probably use diff tool in your IDE or something.

Task: briefly describe the differences made to construct supernet.

Differences from resnet.py to construct search space:

#### 1. Added new argument to BasicBlock. Now it has outplanes parameter.

The block basically consisted of two consecutive 3x3 convolutions:

```
input: x

y = Conv3x3(n1, n2)(x)
y = BatchNorm(y)
y = ReLU(y)
y = Conv3x3(n2, n2)(y)
y = BatchNorm(y)

y = ReLU(y + x)

return y
```

With the new modification, we can choose the number of channels between these two convolutions:

```
y = Conv3x3(n1, n3)(x)
...
y = Conv3x3(n3, n2)(y)
```

#### 2. Replaced all BasicBlock instances in ResNet18 with SearchBlock.

The `for` loop is used to construct multiple BasicBlock options and put them inside the list of available operations. Then this list is used to initialize SearchBlock which is used instead of the default BasicBlock.

All of the arguments for constructing BasicBlock within one SearchBlock are the same except `outplanes` which is scaled by different `channel_multiplier`s.

Note that `downsample` operation now is created separately for each search option and is stored inside each search option.

In [5]:
from supernet import supernet18

In [6]:
# Select suitable device.
# You should probably use either cuda (NVidia GPU) or mps (Apple) backend.
device = torch.device('cuda:0')

In [7]:
# Using the same channel multipliers as in lecture: 2x less channels, same channels, 2x more channels
channel_multipliers = (0.5, 1.0, 2.0)

In [8]:
supernet = supernet18(num_classes=10, zero_init_residual=True, channel_multipliers=channel_multipliers)
supernet.to(device=device)

Supernet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): SearchBlock(
      (ops): ModuleList(
        (0): BasicBlock(
          (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05,

### Define loss function

In [9]:
criterion = nn.CrossEntropyLoss()

### Select hyperparameters

In [10]:
# The same hyperparameters as for training baseline model, EXCEPT the number of epochs, which
# is increased 6 times (120 instead of 20).

lr = 0.25
weight_decay = 5e-4
momentum = 0.9
n_epochs = 120

### Build optimizer and scheduler

In [11]:
optimizer = optim.SGD(supernet.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_dataloader) * n_epochs)

### Define training function

In [12]:
def pretrain_one_epoch(
        model: nn.Module,
        criterion: nn.Module,
        dataloader: DataLoader,
        optimizer: optim.Optimizer,
        scheduler,
        device: torch.device,
        epoch: int,
) -> Tuple[float, float]:
    model.train()

    total_loss = 0.0
    total_correct = 0.0
    total_samples = 0

    wrapped_dataloader = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (inputs, labels) in wrapped_dataloader:
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)

        optimizer.zero_grad()

        # For each search block in the search space, we set only one operation to participate
        # in supernet forward. Only its weights (among other ops from this search block) will
        # be updated using this batch data.
        model.sample_random_architecture()  # <-------------- THIS WAS ADDED

        logits = model(inputs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        with torch.no_grad():
            _, predicted_labels = torch.max(logits, 1)
            total_loss += loss.item()
            total_correct += (predicted_labels == labels).sum().item()
            total_samples += labels.shape[0]

        wrapped_dataloader.set_description(
            f'(train) Epoch={epoch}, lr={scheduler.get_last_lr()[0]:.4f} loss={total_loss / (i + 1):.3f}'
        )

    return total_loss / len(dataloader), total_correct / total_samples


@torch.no_grad()
def validate_one_epoch(
        model: nn.Module,
        criterion: nn.Module,
        dataloader: DataLoader,
        device: torch.device,
        epoch: int,
) -> Tuple[float, float]:
    model.eval()

    total_loss = 0.0
    total_correct = 0.0
    total_samples = 0

    wrapped_dataloader = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (inputs, labels) in wrapped_dataloader:
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)

        logits = model(inputs)
        loss = criterion(logits, labels)
        _, predicted_labels = torch.max(logits, 1)
        total_loss += loss.item()
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.shape[0]

        wrapped_dataloader.set_description(f'(val) Epoch={epoch}, loss={total_loss / (i + 1):.3f}')

    return total_loss / len(dataloader), total_correct / total_samples

### Run training

In [13]:
for epoch in range(n_epochs):
    print(f'Epoch: {epoch}')
    loss, accuracy = pretrain_one_epoch(supernet, criterion, train_dataloader, optimizer, scheduler, device, epoch)
    print(f'train_loss={loss:.4f}, train_accuracy={accuracy:.3%}')
    loss, accuracy = validate_one_epoch(supernet, criterion, test_dataloader, device, epoch)
    print(f'test_loss={loss:.4f}, test_accuracy={accuracy:.3%}')

Epoch: 0


(train) Epoch=0, lr=0.2500 loss=3.953: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 97/97 [00:05<00:00, 18.72it/s]

train_loss=3.9527, train_accuracy=11.242%



(val) Epoch=0, loss=9.321: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 38.26it/s]

test_loss=9.3207, test_accuracy=8.860%
Epoch: 1


...

test_loss=0.4785, test_accuracy=84.480%
Epoch: 119



(train) Epoch=119, lr=0.0000 loss=0.345: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 97/97 [00:02<00:00, 33.89it/s]

train_loss=0.3450, train_accuracy=88.060%



(val) Epoch=119, loss=0.468: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 50.69it/s]

test_loss=0.4681, test_accuracy=84.260%


### Save supernet weights

In [14]:
torch.save(supernet.state_dict(), 'supernet_base_120.pth')